# Including decay in the effective TLS

Note that this is not the same as decay from the third level before the adiabatic eliminiation

In [1]:
import sympy
from sympy import Symbol, symbols, sqrt
from qnet.algebra.circuit_algebra import (
    connect, CircuitSymbol, SLH, move_drive_to_H)
from qnet.algebra.hilbert_space_algebra import LocalSpace
from qnet.algebra.operator_algebra import Destroy, LocalSigma
from qnet.algebra.circuit_algebra import identity_matrix
from qnet.circuit_components.beamsplitter_cc import Beamsplitter
from qnet.circuit_components.displace_cc import Displace


def node_symbol(node_index):
    """Symbolic node in the circuit"""
    return CircuitSymbol("Node_%d" % node_index, cdim=1)


def dagger(op):
    return op.adjoint()


def syms_ops(node_index, n_cavity):
    """Define symbols and operators for a single node, required to write the
    SLH for a single node"""
    HilAtom = LocalSpace('q%d' % int(node_index), basis=('g', 'e'),
                         order_index=(2*node_index))
    HilCavity = LocalSpace('c%d' % int(node_index), dimension=n_cavity,
                           order_index=(2*node_index+1))
    Sym = {}
    Sym['Delta'] = symbols(r'Delta_%s' % node_index, real=True)
    Sym['g'] = symbols(r'g_%s' % node_index, positive=True)
    Sym['Omega'] = symbols(r'Omega_%s' % node_index)
    Sym['I'] = sympy.I
    Sym['kappa'] = symbols(r'kappa', positive=True)
    Sym['gamma'] = symbols(r'gamma', positive=True)
    Op = {}
    Op['a'] = Destroy(hs=HilCavity)
    Op['|g><g|'] = LocalSigma('g', 'g', hs=HilAtom)
    Op['|e><e|'] = LocalSigma('e', 'e', hs=HilAtom)
    Op['|e><g|'] = LocalSigma('e', 'g', hs=HilAtom)
    Op['|g><e|'] = LocalSigma('g', 'e', hs=HilAtom)
    return Sym, Op


def node_hamiltonian(Sym, Op):
    """Symbolic Hamiltonian for a single node, in the RWA"""
    # Symbols
    Δ, g, Ω, I = (Sym['Delta'], Sym['g'], Sym['Omega'], Sym['I'])
    δ = g**2 / Δ
    # Cavity operators
    Op_a = Op['a']; Op_a_dag = dagger(Op_a); Op_n = Op_a_dag * Op_a
    # Qubit operators
    Op_gg = Op['|g><g|']; Op_eg = Op['|e><g|']; Op_ge = dagger(Op_eg)
    # Hamiltonian
    H = -δ * Op_n + (g**2/Δ) * Op_n * Op_gg \
        -I * (g / (2*Δ)) * Ω * (Op_eg*Op_a - Op_ge*Op_a_dag)
    return H


def node_slh(node_index, n_cavity):
    """SLH description for a single node with the given `node_index` (which
    will become the subscript in all symbols) and `n_cavity` number of levels
    for the cavity
    """
    Sym, Op = syms_ops(node_index, n_cavity)
    S = identity_matrix(2)
    κ = Sym['kappa']
    γ = Sym['gamma']
    L = [sqrt(2 * κ) * Op['a'], sqrt(2 * γ) * Op['|g><e|']]
    H = node_hamiltonian(Sym, Op)
    return SLH(S, L, H)


def network_circuit(n_nodes, topology='open'):
    """Construct the network with the given topology"""
    if topology not in ['open', 'bs_fb', 'driven_bs_fb', 'driven_open']:
        raise ValueError("Unknown topology: %s" % topology)
    nodes = []
    connections = []
    prev_node = None
    for i in range(n_nodes):
        ind = i + 1
        cur_node = node_symbol(ind)
        nodes.append(cur_node)
        if prev_node is not None:
            connections.append(((prev_node, 0), (cur_node, 0)))
        prev_node = cur_node
    if 'bs_fb' in topology:
        BS = Beamsplitter('BS', theta=symbols('theta', real=True))
        nodes.append(BS)
        connections.append(((prev_node, 0), (BS, 0)))
        connections.append(((BS, 1), (nodes[0], 0)))
        if 'driven' in topology:
            W = Displace('W', alpha=symbols('alpha'))
            nodes.append(W)
            connections.append(((W, 0), (BS, 1)))
    else: # open topology
        if 'driven' in topology:  # driven_open
            W = Displace('W', alpha=symbols('alpha'))
            nodes.append(W)
            connections.append(((W, 0), (nodes[0], 0)))

    circuit = connect(nodes, connections, force_SLH=False)
    return circuit


def network_slh(n_cavity, n_nodes, topology='open', inhom=False):
    """Return the symbolic SLH for the entire network"""
    circuit = network_circuit(n_nodes, topology)
    slh_mapping = {}
    for i in range(n_nodes):
        ind = i + 1  # 1-based indexing of nodes
        slh_mapping[node_symbol(ind)] = node_slh(ind, n_cavity)
    S, L, H = circuit.substitute(slh_mapping).toSLH()
    κ = Symbol('kappa', positive=True)
    α, Ω = symbols('alpha Omega_alpha')
    λ = symbols('lambda', real=True)
    S.expand().simplify_scalar()
    H = H.expand().simplify_scalar().substitute({
        α: (λ * Ω.conjugate() / (2 * sqrt(κ))),
    })
    L = L.expand().simplify_scalar().substitute({
        α: (λ * Ω.conjugate() / (2 * sqrt(κ))),
    })
    slh = SLH(S, L, H)
    if not inhom:
        slh = move_drive_to_H(slh)
    slh.n_cavity = n_cavity
    slh.n_nodes = n_nodes
    return slh


In [2]:
slh = network_slh(n_cavity=2, n_nodes=2)

In [4]:
slh.L

[[√2 √κ â^(c₁) + √2 √κ â^(c₂)], [√2 √γ |g⟩⟨e|^(q₁) + √2 √γ |g⟩⟨e|^(q₂)]]

In [3]:
slh.H

g₁²/Δ₁ |g⟩⟨g|^(q₁) â^(c₁)† â^(c₁) - ⅈ Ω₁ g₁/(2 Δ₁) |e⟩⟨g|^(q₁) â^(c₁) + ⅈ Ω₁ g₁/(2 Δ₁) |g⟩⟨e|^(q₁) â^(c₁)† + ⅈ γ |e⟩⟨g|^(q₁) |g⟩⟨e|^(q₂) - ⅈ γ |g⟩⟨e|^(q₁) |e⟩⟨g|^(q₂) - g₁²/Δ₁ â^(c₁)† â^(c₁) + ⅈ κ â^(c₁)† â^(c₂) - ⅈ κ â^(c₁) â^(c₂)† + g₂²/Δ₂ |g⟩⟨g|^(q₂) â^(c₂)† â^(c₂) - ⅈ Ω₂ g₂/(2 Δ₂) |e⟩⟨g|^(q₂) â^(c₂) + ⅈ Ω₂ g₂/(2 Δ₂) |g⟩⟨e|^(q₂) â^(c₂)† - g₂²/Δ₂ â^(c₂)† â^(c₂)